In [30]:
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')
import os
import pandas as pd
import os.path
from os import path
import datetime 

In [36]:
from urllib.request import urlopen
import json

with open('../data/raw/geojson-counties-fips.json') as json_file:
    counties = json.load(json_file)

In [37]:
import plotly.express as px

In [38]:
import csv
countyAndState_ToFips = {}

with open('../data/raw/CountyToFips.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            county = row[2]
            indexOfP = county.find("Parish")
            if indexOfP != -1:
                county = county[:indexOfP] + "County"
                #print(county)
            state = row[1]
            print(state)
            countyAndState_ToFips[(county, row[1])] = row[0]
            line_count += 1
    print(f'Processed {line_count} lines.')
print(countyAndState_ToFips)




    

Column names are FIPS, STNAME, CTYNAME
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Indiana
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Iowa
Io

Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Texas
Utah
Utah
Utah
Utah
Utah
Utah
Utah
Utah
Utah
Utah
Utah
Utah
Utah
Utah
Utah
Utah
Utah
Utah
Utah
Utah
Utah
Utah
Utah
Utah
Utah
Utah
Utah
Utah
Utah
Vermont
Vermont
Vermont
Vermont
Vermont
Vermont
Vermont
Vermont
Vermont
Vermont
Vermont
Vermont
Vermont
Vermont
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Virginia
Vi

In [39]:
dateToDictOf_CountyToAnomalyScore = {}
with open('../data/processed/AnomaliesTimespreadByIsoForest.csv') as csv_file1:
    currentDate_CountyToAnomalyScore = {}
    currentDate = -1
    
    csv_reader = csv.reader(csv_file1, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            if currentDate == -1:
                currentDate = row[0]
                
            if currentDate != row[0]:
                dateToDictOf_CountyToAnomalyScore[currentDate] = currentDate_CountyToAnomalyScore
                currentDate_CountyToAnomalyScore = {}
                currentDate = row[0]
            
            #if row[1] in countyToFips:
            county = row[1]
            state = row[2]
            if county == "New York City County":
                county = "New York County"
            if county != "Unknown County":
                currentDate_CountyToAnomalyScore[countyAndState_ToFips[(county, state)]] = row[8]

            line_count += 1
    print(f'Processed {line_count} lines.')
print(dateToDictOf_CountyToAnomalyScore)

Column names are date, county, state, pop, cases, deaths, casesPerCapita, deathsPerCapita, scores, anomaly
Processed 6907 lines.
{'43890': {'53033': '-0.09108154744998398'}, '43891': {'53033': '-0.10954625829766607'}, '43892': {'53033': '-0.13730696853697977'}, '43893': {'53033': '-0.14657558847694696'}, '43894': {'53033': '-0.02827063641783667'}, '43895': {'53033': '-0.035127408269305715'}, '43896': {'53033': '-0.054662074163758234'}, '43897': {'53033': '-0.06348646394618906'}, '43898': {'53033': '-0.08860097844939008', '12113': '-0.0077639981856446605'}, '43899': {'53033': '-0.13915870550194154'}, '43900': {'53033': '-0.13159634766942985'}, '43901': {'53033': '-0.14285808853037407'}, '43902': {'53033': '-0.1665307241254862', '20209': '-0.03681494559897969'}, '43903': {'53033': '-0.14519391285182526', '12113': '-0.014366621846310212', '6085': '-0.08243367038295557', '20209': '-0.0017330709448348802'}, '43904': {'53033': '-0.16656754435247734', '12113': '-0.06602145110965041', '6085': 

In [40]:
fileNumber = 0
for cDate, cDictOfCountyToAnomalyScore in dateToDictOf_CountyToAnomalyScore.items():
    outputFile = '../data/processed/AnomalyAnimationData/iso' + str(fileNumber) + '.csv'
    if path.exists(outputFile):
        os.remove(outputFile)
    with open(outputFile, 'a', newline='') as lof_file:
        fieldnames = ['fips', 'anomalyScore']
        writer = csv.DictWriter(lof_file, fieldnames=fieldnames)
        myHeaderRowData = {}
        for colName in fieldnames:
            myHeaderRowData[colName] = colName
        writer.writerow(myHeaderRowData)



        rowN = 0
        for cCountyAndState, cFips in countyAndState_ToFips.items():
            rowN = rowN + 1
            myRowData = {}
            if cFips in cDictOfCountyToAnomalyScore:
                if len(cFips) == 4:
                    myRowData['fips'] = "0" + str(cFips)
                else:
                    myRowData['fips'] = cFips
                myRowData['anomalyScore'] = cDictOfCountyToAnomalyScore[cFips]
            else:
                if len(cFips) == 4:
                    myRowData['fips'] = "0" + str(cFips)
                else:
                    myRowData['fips'] = cFips
                myRowData['anomalyScore'] = 0
            writer.writerow(myRowData)
        #print(rowN)
    fileNumber = fileNumber + 1

3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143
3143


In [50]:
start_date = datetime.datetime(2020, 2, 29)
curr_date = start_date
# iterate through all anomaly date data
for i in range(0, 248):
    curr_date = start_date + datetime.timedelta(days=i)
    df = pd.read_csv("../data/processed/AnomalyAnimationData/iso" + str(i) + ".csv",
                       dtype={"fips": str})

    df = df.sort_values(by=['fips'])


    fig = px.choropleth_mapbox(df, geojson=counties, locations='fips', color='anomalyScore',
                               color_continuous_scale="Viridis",
                               range_color=(df["anomalyScore"].min(), 0),
                               mapbox_style="carto-positron",
                               zoom=2.8, center = {"lat": 37.0902, "lon": -95.7129},
                               opacity=0.5,
                               labels={'anomalyScore':'Anomaly Score'}
                              )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    if not os.path.exists("../data/processed/AnomalyAnimationImages"):
        os.mkdir("../data/processed/AnomalyAnimationImages")
    # place the date of the data as the figure name
    filename = curr_date.strftime("../data/processed/AnomalyAnimationImages/%m-%d-%Y.png")
    if not os.path.exists(filename)
        print(filename)
        fig.write_image(filename)
        

../data/processed/AnomalyAnimationImages/02-29-2020.png
../data/processed/AnomalyAnimationImages/03-01-2020.png
../data/processed/AnomalyAnimationImages/03-02-2020.png
../data/processed/AnomalyAnimationImages/03-03-2020.png
../data/processed/AnomalyAnimationImages/03-04-2020.png
../data/processed/AnomalyAnimationImages/03-05-2020.png
../data/processed/AnomalyAnimationImages/03-06-2020.png
../data/processed/AnomalyAnimationImages/03-07-2020.png
../data/processed/AnomalyAnimationImages/03-08-2020.png
../data/processed/AnomalyAnimationImages/03-09-2020.png
../data/processed/AnomalyAnimationImages/03-10-2020.png
../data/processed/AnomalyAnimationImages/03-11-2020.png
../data/processed/AnomalyAnimationImages/03-12-2020.png
../data/processed/AnomalyAnimationImages/03-13-2020.png
../data/processed/AnomalyAnimationImages/03-14-2020.png
../data/processed/AnomalyAnimationImages/03-15-2020.png
../data/processed/AnomalyAnimationImages/03-16-2020.png
../data/processed/AnomalyAnimationImages/03-17-2

../data/processed/AnomalyAnimationImages/07-25-2020.png
../data/processed/AnomalyAnimationImages/07-26-2020.png
../data/processed/AnomalyAnimationImages/07-27-2020.png
../data/processed/AnomalyAnimationImages/07-28-2020.png
../data/processed/AnomalyAnimationImages/07-29-2020.png
../data/processed/AnomalyAnimationImages/07-30-2020.png
../data/processed/AnomalyAnimationImages/07-31-2020.png
../data/processed/AnomalyAnimationImages/08-01-2020.png
../data/processed/AnomalyAnimationImages/08-02-2020.png
../data/processed/AnomalyAnimationImages/08-03-2020.png
../data/processed/AnomalyAnimationImages/08-04-2020.png
../data/processed/AnomalyAnimationImages/08-05-2020.png
../data/processed/AnomalyAnimationImages/08-06-2020.png
../data/processed/AnomalyAnimationImages/08-07-2020.png
../data/processed/AnomalyAnimationImages/08-08-2020.png
../data/processed/AnomalyAnimationImages/08-09-2020.png
../data/processed/AnomalyAnimationImages/08-10-2020.png
../data/processed/AnomalyAnimationImages/08-11-2

3143

-0.10162013390758806

In [26]:
dateToDictOf_CountyToAnomalyScore = {}
with open('../data/processed/CasesAnomaliesTimespreadByIsoForest.csv') as csv_file1:
    currentDate_CountyToAnomalyScore = {}
    currentDate = -1
    
    csv_reader = csv.reader(csv_file1, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            if currentDate == -1:
                currentDate = row[0]
                
            if currentDate != row[0]:
                dateToDictOf_CountyToAnomalyScore[currentDate] = currentDate_CountyToAnomalyScore
                currentDate_CountyToAnomalyScore = {}
                currentDate = row[0]
            
            #if row[1] in countyToFips:
            county = row[1]
            state = row[2]
            if county == "New York City County":
                county = "New York County"
            if county != "Unknown County":
                currentDate_CountyToAnomalyScore[countyAndState_ToFips[(county, state)]] = row[8]

            line_count += 1
    print(f'Processed {line_count} lines.')
print(dateToDictOf_CountyToAnomalyScore)

Column names are date, county, state, pop, cases, deaths, casesPerCapita, deathsPerCapita, scores, anomaly
Processed 6921 lines.
{'43855': {'53061': '-0.0035191936044266514'}, '43856': {'53061': '-0.005257503781996742'}, '43857': {'53061': '-0.004393886931044477'}, '43858': {'53061': '-0.00408942172816229'}, '43859': {'53061': '-0.004739323733153555'}, '43860': {'17031': '-0.0026264351704160527'}, '43861': {'6085': '-0.0026418670404732003'}, '43862': {'25025': '-0.013511213987472348'}, '43863': {'25025': '-0.011597729074010465'}, '43864': {'25025': '-0.013370176254775323'}, '43865': {'25025': '-0.012482442673526939'}, '43866': {'25025': '-0.013470785438646549'}, '43867': {'25025': '-0.014264531762015364'}, '43868': {'25025': '-0.013296106886282466'}, '43869': {'25025': '-0.012722483407434315'}, '43870': {'25025': '-0.013675563619714004'}, '43871': {'25025': '-0.015260879700349395'}, '43872': {'25025': '-0.014570269760734278'}, '43873': {'25025': '-0.015292041087638819'}, '43874': {'250

In [27]:
fileNumber = 0
for cDate, cDictOfCountyToAnomalyScore in dateToDictOf_CountyToAnomalyScore.items():
    outputFile = '../data/processed/AnomalyAnimationData/CasesIso' + str(fileNumber) + '.csv'
    if path.exists(outputFile):
        os.remove(outputFile)
    with open(outputFile, 'a', newline='') as lof_file:
        fieldnames = ['fips', 'anomalyScore']
        writer = csv.DictWriter(lof_file, fieldnames=fieldnames)
        myHeaderRowData = {}
        for colName in fieldnames:
            myHeaderRowData[colName] = colName
        writer.writerow(myHeaderRowData)



        rowN = 0
        for cCountyAndState, cFips in countyAndState_ToFips.items():
            rowN = rowN + 1
            myRowData = {}
            if cFips in cDictOfCountyToAnomalyScore:
                if len(cFips) == 4:
                    myRowData['fips'] = "0" + str(cFips)
                else:
                    myRowData['fips'] = cFips
                myRowData['anomalyScore'] = cDictOfCountyToAnomalyScore[cFips]
            else:
                if len(cFips) == 4:
                    myRowData['fips'] = "0" + str(cFips)
                else:
                    myRowData['fips'] = cFips
                myRowData['anomalyScore'] = 0
            writer.writerow(myRowData)
        #print(rowN)
    fileNumber = fileNumber + 1

In [32]:
start_date = datetime.datetime(2020, 2, 29)
curr_date = start_date
# iterate through all anomaly date data
for i in range(0, 248):
    curr_date = start_date + datetime.timedelta(days=i)
    df = pd.read_csv("../data/processed/AnomalyAnimationData/CasesIso" + str(i) + ".csv",
                       dtype={"fips": str})

    df = df.sort_values(by=['fips'])


    fig = px.choropleth_mapbox(df, geojson=counties, locations='fips', color='anomalyScore',
                               color_continuous_scale="Viridis",
                               range_color=(df["anomalyScore"].min(), 0),
                               mapbox_style="carto-positron",
                               zoom=2.8, center = {"lat": 37.0902, "lon": -95.7129},
                               opacity=0.5,
                               labels={'anomalyScore':'Anomaly Score'}
                              )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    if not os.path.exists("../data/processed/CasesIsoAnomalyAnimationImages"):
        os.mkdir("../data/processed/CasesIsoAnomalyAnimationImages")
    # place the date of the data as the figure name
    filename = curr_date.strftime("../data/processed/CasesIsoAnomalyAnimationImages/%m-%d-%Y.png")
    if not os.path.exists(filename):
        print(filename)
        fig.write_image(filename)
        

../data/processed/CasesIsoAnomalyAnimationImages/02-29-2020.png
../data/processed/CasesIsoAnomalyAnimationImages/03-01-2020.png
../data/processed/CasesIsoAnomalyAnimationImages/03-02-2020.png
../data/processed/CasesIsoAnomalyAnimationImages/03-03-2020.png
../data/processed/CasesIsoAnomalyAnimationImages/03-04-2020.png
../data/processed/CasesIsoAnomalyAnimationImages/03-05-2020.png
../data/processed/CasesIsoAnomalyAnimationImages/03-06-2020.png
../data/processed/CasesIsoAnomalyAnimationImages/03-07-2020.png
../data/processed/CasesIsoAnomalyAnimationImages/03-08-2020.png
../data/processed/CasesIsoAnomalyAnimationImages/03-09-2020.png
../data/processed/CasesIsoAnomalyAnimationImages/03-10-2020.png
../data/processed/CasesIsoAnomalyAnimationImages/03-11-2020.png
../data/processed/CasesIsoAnomalyAnimationImages/03-12-2020.png
../data/processed/CasesIsoAnomalyAnimationImages/03-13-2020.png
../data/processed/CasesIsoAnomalyAnimationImages/03-14-2020.png
../data/processed/CasesIsoAnomalyAnimati

../data/processed/CasesIsoAnomalyAnimationImages/07-07-2020.png
../data/processed/CasesIsoAnomalyAnimationImages/07-08-2020.png
../data/processed/CasesIsoAnomalyAnimationImages/07-09-2020.png
../data/processed/CasesIsoAnomalyAnimationImages/07-10-2020.png
../data/processed/CasesIsoAnomalyAnimationImages/07-11-2020.png
../data/processed/CasesIsoAnomalyAnimationImages/07-12-2020.png
../data/processed/CasesIsoAnomalyAnimationImages/07-13-2020.png
../data/processed/CasesIsoAnomalyAnimationImages/07-14-2020.png
../data/processed/CasesIsoAnomalyAnimationImages/07-15-2020.png
../data/processed/CasesIsoAnomalyAnimationImages/07-16-2020.png
../data/processed/CasesIsoAnomalyAnimationImages/07-17-2020.png
../data/processed/CasesIsoAnomalyAnimationImages/07-18-2020.png
../data/processed/CasesIsoAnomalyAnimationImages/07-19-2020.png
../data/processed/CasesIsoAnomalyAnimationImages/07-20-2020.png
../data/processed/CasesIsoAnomalyAnimationImages/07-21-2020.png
../data/processed/CasesIsoAnomalyAnimati

In [37]:
dateToDictOf_CountyToAnomalyScore = {}
with open('../data/processed/AnomaliesTimespreadByLocalOutlier.csv') as csv_file1:
    currentDate_CountyToAnomalyScore = {}
    currentDate = -1
    
    csv_reader = csv.reader(csv_file1, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            if currentDate == -1:
                currentDate = row[0]
                
            if currentDate != row[0]:
                dateToDictOf_CountyToAnomalyScore[currentDate] = currentDate_CountyToAnomalyScore
                currentDate_CountyToAnomalyScore = {}
                currentDate = row[0]
            
            #if row[1] in countyToFips:
            county = row[1]
            state = row[2]
            if county == "New York City County":
                county = "New York County"
            if county != "Unknown County":
                currentDate_CountyToAnomalyScore[countyAndState_ToFips[(county, state)]] = row[8]

            line_count += 1
    print(f'Processed {line_count} lines.')
print(dateToDictOf_CountyToAnomalyScore)

Column names are date, county, state, pop, cases, deaths, casesPerCapita, deathsPerCapita, scores, anomaly
Processed 1 lines.
{}


In [38]:
fileNumber = 0
for cDate, cDictOfCountyToAnomalyScore in dateToDictOf_CountyToAnomalyScore.items():
    outputFile = '../data/processed/AnomalyAnimationData/LOF' + str(fileNumber) + '.csv'
    if path.exists(outputFile):
        os.remove(outputFile)
    with open(outputFile, 'a', newline='') as lof_file:
        fieldnames = ['fips', 'anomalyScore']
        writer = csv.DictWriter(lof_file, fieldnames=fieldnames)
        myHeaderRowData = {}
        for colName in fieldnames:
            myHeaderRowData[colName] = colName
        writer.writerow(myHeaderRowData)



        rowN = 0
        for cCountyAndState, cFips in countyAndState_ToFips.items():
            rowN = rowN + 1
            myRowData = {}
            if cFips in cDictOfCountyToAnomalyScore:
                if len(cFips) == 4:
                    myRowData['fips'] = "0" + str(cFips)
                else:
                    myRowData['fips'] = cFips
                myRowData['anomalyScore'] = cDictOfCountyToAnomalyScore[cFips]
            else:
                if len(cFips) == 4:
                    myRowData['fips'] = "0" + str(cFips)
                else:
                    myRowData['fips'] = cFips
                myRowData['anomalyScore'] = 0
            writer.writerow(myRowData)
        #print(rowN)
    fileNumber = fileNumber + 1

In [39]:
start_date = datetime.datetime(2020, 2, 29)
curr_date = start_date
# iterate through all anomaly date data
for i in range(0, 248):
    curr_date = start_date + datetime.timedelta(days=i)
    df = pd.read_csv("../data/processed/AnomalyAnimationData/LOF" + str(i) + ".csv",
                       dtype={"fips": str})

    df = df.sort_values(by=['fips'])


    fig = px.choropleth_mapbox(df, geojson=counties, locations='fips', color='anomalyScore',
                               color_continuous_scale="Viridis",
                               range_color=(df["anomalyScore"].min(), 0),
                               mapbox_style="carto-positron",
                               zoom=2.8, center = {"lat": 37.0902, "lon": -95.7129},
                               opacity=0.5,
                               labels={'anomalyScore':'Anomaly Score'}
                              )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    if not os.path.exists("../data/processed/LOFAnomalyAnimationImages"):
        os.mkdir("../data/processed/LOFAnomalyAnimationImages")
    # place the date of the data as the figure name
    filename = curr_date.strftime("../data/processed/LOFAnomalyAnimationImages/%m-%d-%Y.png")
    if not os.path.exists(filename):
        print(filename)
        fig.write_image(filename)
        

FileNotFoundError: [Errno 2] File ../data/processed/AnomalyAnimationData/LOF0.csv does not exist: '../data/processed/AnomalyAnimationData/LOF0.csv'

In [33]:
dateToDictOf_CountyToAnomalyScore = {}
with open('../data/processed/CasesAnomaliesTimespreadByLocalOutlier.csv') as csv_file1:
    currentDate_CountyToAnomalyScore = {}
    currentDate = -1
    
    csv_reader = csv.reader(csv_file1, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            if currentDate == -1:
                currentDate = row[0]
                
            if currentDate != row[0]:
                dateToDictOf_CountyToAnomalyScore[currentDate] = currentDate_CountyToAnomalyScore
                currentDate_CountyToAnomalyScore = {}
                currentDate = row[0]
            
            #if row[1] in countyToFips:
            county = row[1]
            state = row[2]
            if county == "New York City County":
                county = "New York County"
            if county != "Unknown County":
                currentDate_CountyToAnomalyScore[countyAndState_ToFips[(county, state)]] = row[8]

            line_count += 1
    print(f'Processed {line_count} lines.')
print(dateToDictOf_CountyToAnomalyScore)

Column names are date, county, state, pop, cases, deaths, casesPerCapita, deathsPerCapita, scores, anomaly
Processed 2692 lines.
{'43919': {'36103': '-0.005217835217106281'}, '43920': {'36103': '-0.025908086316277945'}, '43921': {'36103': '-0.0394611209283009'}, '43922': {'36103': '-0.05491518065359291'}, '43923': {'36103': '-0.07464904507089576'}, '43924': {'36103': '-0.08152323572806053'}, '43925': {'36103': '-0.09310497667510986'}, '43926': {'36103': '-0.093688187856525'}, '43927': {'36103': '-0.09485579466517857'}, '43928': {'36103': '-0.0960249825168793'}, '43929': {'36103': '-0.09778173343409458'}, '43930': {'36103': '-0.09954205375348602'}, '43931': {'36103': '-0.09954205375348602', '25025': '-0.003126463799635548'}, '43932': {'36103': '-0.09954205375348602', '25025': '-0.005217835217106281'}, '43933': {'36103': '-0.10248387266180992', '25025': '-0.012054014079519337'}, '43934': {'36103': '-0.10248387266180992', '25025': '-0.01576001470111088'}, '43935': {'36103': '-0.1024838726

In [34]:
fileNumber = 0
for cDate, cDictOfCountyToAnomalyScore in dateToDictOf_CountyToAnomalyScore.items():
    outputFile = '../data/processed/AnomalyAnimationData/CasesLOF' + str(fileNumber) + '.csv'
    if path.exists(outputFile):
        os.remove(outputFile)
    with open(outputFile, 'a', newline='') as lof_file:
        fieldnames = ['fips', 'anomalyScore']
        writer = csv.DictWriter(lof_file, fieldnames=fieldnames)
        myHeaderRowData = {}
        for colName in fieldnames:
            myHeaderRowData[colName] = colName
        writer.writerow(myHeaderRowData)



        rowN = 0
        for cCountyAndState, cFips in countyAndState_ToFips.items():
            rowN = rowN + 1
            myRowData = {}
            if cFips in cDictOfCountyToAnomalyScore:
                if len(cFips) == 4:
                    myRowData['fips'] = "0" + str(cFips)
                else:
                    myRowData['fips'] = cFips
                myRowData['anomalyScore'] = cDictOfCountyToAnomalyScore[cFips]
            else:
                if len(cFips) == 4:
                    myRowData['fips'] = "0" + str(cFips)
                else:
                    myRowData['fips'] = cFips
                myRowData['anomalyScore'] = 0
            writer.writerow(myRowData)
        #print(rowN)
    fileNumber = fileNumber + 1

In [36]:
start_date = datetime.datetime(2020, 2, 29)
curr_date = start_date
# iterate through all anomaly date data
for i in range(0, 248):
    curr_date = start_date + datetime.timedelta(days=i)
    df = pd.read_csv("../data/processed/AnomalyAnimationData/CasesLOF" + str(i) + ".csv",
                       dtype={"fips": str})

    df = df.sort_values(by=['fips'])


    fig = px.choropleth_mapbox(df, geojson=counties, locations='fips', color='anomalyScore',
                               color_continuous_scale="Viridis",
                               range_color=(df["anomalyScore"].min(), 0),
                               mapbox_style="carto-positron",
                               zoom=2.8, center = {"lat": 37.0902, "lon": -95.7129},
                               opacity=0.5,
                               labels={'anomalyScore':'Anomaly Score'}
                              )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    if not os.path.exists("../data/processed/CasesLOFAnomalyAnimationImages"):
        os.mkdir("../data/processed/CasesLOFAnomalyAnimationImages")
    # place the date of the data as the figure name
    filename = curr_date.strftime("../data/processed/CasesLOFAnomalyAnimationImages/%m-%d-%Y.png")
    if not os.path.exists(filename):
        print(filename)
        fig.write_image(filename)
        

../data/processed/CasesLOFAnomalyAnimationImages/09-10-2020.png
../data/processed/CasesLOFAnomalyAnimationImages/09-11-2020.png
../data/processed/CasesLOFAnomalyAnimationImages/09-12-2020.png
../data/processed/CasesLOFAnomalyAnimationImages/09-13-2020.png
../data/processed/CasesLOFAnomalyAnimationImages/09-14-2020.png
../data/processed/CasesLOFAnomalyAnimationImages/09-15-2020.png
../data/processed/CasesLOFAnomalyAnimationImages/09-16-2020.png
../data/processed/CasesLOFAnomalyAnimationImages/09-17-2020.png
../data/processed/CasesLOFAnomalyAnimationImages/09-18-2020.png
../data/processed/CasesLOFAnomalyAnimationImages/09-19-2020.png
../data/processed/CasesLOFAnomalyAnimationImages/09-20-2020.png
../data/processed/CasesLOFAnomalyAnimationImages/09-21-2020.png
../data/processed/CasesLOFAnomalyAnimationImages/09-22-2020.png
../data/processed/CasesLOFAnomalyAnimationImages/09-23-2020.png
../data/processed/CasesLOFAnomalyAnimationImages/09-24-2020.png
../data/processed/CasesLOFAnomalyAnimati

FileNotFoundError: [Errno 2] File ../data/processed/AnomalyAnimationData/CasesLOF220.csv does not exist: '../data/processed/AnomalyAnimationData/CasesLOF220.csv'